In [1]:
# the selenium part is took from the below link. it is helpful to understand what you need for scraping by Selenium.
# https://medium.freecodecamp.org/better-web-scraping-in-python-with-selenium-beautiful-soup-and-pandas-d6390592e251

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

In [2]:
import time
import requests

url = "https://mass511.com/"
res=requests.get(url)
res.status_code


200

## 1- MANUAL CHECK:

### entering the origins&destinations to website

In [3]:
# for manual check if code runs:
origin='42.436659, -71.090072'
destination='42.411050, -71.092265'

In [4]:
# this cell is took from the abovementioned link:
# open the webpage by pyhton code!

driver = webdriver.Chrome(executable_path='./chromedriver 2')
driver.implicitly_wait(30)
driver.get(url)

In [7]:
enter_A_button = driver.find_element_by_id('startLocationText') # entering origin point to box A
enter_B_button = driver.find_element_by_id('endLocationText') # entering  destination point to box B
drive_button =driver.find_element_by_id('generateRouteBtn') # clicking the 'Drive' button and open 1st Route.
clear_button =driver.find_element_by_id('clearRouteBtn') # clear the A and B Boxes for the next query!

In [8]:
enter_A_button.send_keys(origin)
enter_B_button.send_keys(destination)
drive_button.click()

In [29]:
clear_button.click()

In [5]:
driver.quit()

## 2- ORIGIN & DESTINATIONS:

In [43]:
x_y=pd.read_csv('./Data/routes_with_zones.csv')
data = x_y[x_y.Time == "8:30"]
data.head()

,Unnamed: 0,osmid_start,x_start,y_start,start_zone,osmid_end,x_end,y_end,end_zone,Time
0,0,66450694,-71.090072,42.436659,1,66449255,-71.092265,42.411050,2,8:30
3,3,66436104,-71.088950,42.439999,1,66445131,-71.090880,42.411045,2,8:30
6,6,66483089,-71.094324,42.430675,1,3191029804,-71.094565,42.406251,2,8:30
9,9,66474562,-71.075071,42.404712,2,66422683,-71.127038,42.410869,0,8:30
12,12,66481516,-71.088709,42.433929,1,66461868,-71.088943,42.404121,2,8:30


In [44]:
# change the x with y because of given conversely
new_cols={'x_start':'y_start','y_start':'x_start','x_end':'y_end','y_end':'x_end'}

data.rename(columns=new_cols,inplace=True)
data.head(2)

,Unnamed: 0,osmid_start,y_start,x_start,start_zone,osmid_end,y_end,x_end,end_zone,Time
0,0,66450694,-71.090072,42.436659,1,66449255,-71.092265,42.411050,2,8:30
3,3,66436104,-71.088950,42.439999,1,66445131,-71.090880,42.411045,2,8:30


In [45]:
# create origin & destination coordinates (adresses) in order to enter the A and B Boxes on the web page:
data['origin'] = data.apply(lambda row: str(row['x_start']) +', '+ str(row['y_start']), axis=1)
data['destination'] = data.apply(lambda row: str(row['x_end']) +', '+ str(row['y_end']), axis=1)

/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [46]:
# create flexible number of rows in order to send query to web page. web page doest allow to scrape more than 
# 149 points at each hit! so you would enter the max allowed number. But you can try for more rows and observe
# if works for more e.g., 200 or 500 or 1000 or 3000 or more
data=data[:3] 
data.tail()

,Unnamed: 0,osmid_start,y_start,x_start,start_zone,osmid_end,y_end,x_end,end_zone,Time,origin,destination
0,0,66450694,-71.090072,42.436659,1,66449255,-71.092265,42.411050,2,8:30,"42.436659000000006, -71.09007199999999","42.41105, -71.092265"
3,3,66436104,-71.088950,42.439999,1,66445131,-71.090880,42.411045,2,8:30,"42.4399985, -71.0889505","42.41104489999999, -71.0908802"
6,6,66483089,-71.094324,42.430675,1,3191029804,-71.094565,42.406251,2,8:30,"42.4306754, -71.0943239","42.4062507, -71.09456479999999"


In [47]:
data.shape

(3, 12)

## 3- SCAPE BY LOOP:

In [48]:
cengiz = []
counter=0
driver = webdriver.Chrome(executable_path='./chromedriver')
driver.implicitly_wait(23)
driver.get(url)
soup=BeautifulSoup(driver.page_source, 'lxml')

for i in data.index:
    
    try:
        enter_A_button = driver.find_element_by_id('startLocationText') # entering origin point to box A
        enter_B_button = driver.find_element_by_id('endLocationText') # entering  destination point to box B
        drive_button =driver.find_element_by_id('generateRouteBtn') # clicking the 'Drive' button and open 1st Route.
        clear_button =driver.find_element_by_id('clearRouteBtn') # clear the A and B Boxes for the next query!

        origin=data['origin'][i] # takes the x,y coordinates of the i^th row of the 'origin' column
        destination=data['destination'][i] # takes the x,y coordinates of the i^th row of the 'destination' column 

        enter_A_button.send_keys(origin)
        enter_B_button.send_keys(destination)
        drive_button.click()
        time.sleep(6) # wait time for each query

        driver.implicitly_wait(1) # wait time after clicking the button

        soup=BeautifulSoup(driver.page_source, 'lxml')
        for j in range(2): # there are 2 Routes where maximum possibilites. 'j' is running for routes
            
            if j==0: # 1st Route loop
                try:
                    loc_1=soup.find('div',{'id':'routeTab-1'}) # the location where keeps the all required data for Route 1
                    route1 = {}
                    key = 0 # create keys for 'from to where','time' and 'distance' in the nested dictionary 
                    cengiz.append(route1)
                    for k in loc_1.find_all('span'): # scrape for 'from to where', 'time' and 'distance columns'
                        route1[key]=k.text.split("\n")[0] # if you remove [0] it works but format will change
                        key +=1

                        loc_1_a=loc_1.find('ul',{'class':'list-group'}) #the location where keeps the all breakdown directions(legs)
                        direction={}
                        key2=0 # create kesy for all legs in the 'direction' dictionary
                        for l in loc_1_a: # scrape all given directions: Bethany did this part for me!
                            direct = str(l.br.previous_sibling).strip()
                            direct=direct.lstrip('<b>').rstrip('</br>')
                            direction[key2]=direct
                            key2+=1

                        route1['direction']=direction
                        route1['route']='Route 1'
                        route1['x_start']=data['x_start'][i]
                        route1['y_start']=data['y_start'][i]
                        route1['x_end']=data['x_end'][i]
                        route1['y_end']=data['y_end'][i]
                        route1['origin']=data['origin'][i]
                        route1['destination']=data['destination'][i]
                        route1['start_zone']=data['start_zone'][i]
                        route1['end_zone']=data['end_zone'][i]
                except:
                    pass

                
            elif j==1:# 2nd Route loop
                try: 
                    loc_2=soup.find('div',{'id':'routeTab-2'}) # the location where keeps the all required data for Route 2
                    route2 = {}
                    key = 0
                    cengiz.append(route2)
                    for m in loc_2.find_all('span'):
                        route2[key]=m.text.split("\n")[0] # if you remove [0] it works but format will change
                        key +=1

                        loc_2_a=loc_2.find('ul',{'class':'list-group'})
                        direction={}
                        key2=0
                        for n in loc_2_a:
                            direct = str(n.br.previous_sibling).strip()
                            direct=direct.lstrip('<b>').rstrip('</br>')
                            direction[key2]=direct
                            key2+=1

                        route2['direction']=direction
                        route2['route']='Route 2'
                        route2['x_start']=data['x_start'][i]
                        route2['y_start']=data['y_start'][i]
                        route2['x_end']=data['x_end'][i]
                        route2['y_end']=data['y_end'][i]
                        route2['origin']=data['origin'][i]
                        route2['destination']=data['destination'][i]
                        route2['start_zone']=data['start_zone'][i]
                        route2['end_zone']=data['end_zone'][i]
                except:
                    pass         

        clear_button.click() # click to Reset button
        counter+=1
        print('counter:',counter)
        
    except:
        driver.quit() # close the webpage if the loop failed to scrape while works !
        
driver.quit()# close the webpage after finishing the main loop

counter: 1
counter: 2
counter: 3


In [49]:
cengiz

[{0: 'From 49 Palmer St, Medford, MA 02155, USA to 375 Riverside Ave, Medford, MA 02155, USA',
  'direction': {0: 'From Highland Ave Eastbound to Fellsway E Southbound',
   1: 'From Fellsway E Southbound to Pleasant St Westbound',
   2: 'From Pleasant St Westbound to Salem St Westbound',
   3: 'From Salem St Westbound to Rt-60 Westbound',
   4: 'From Rt-60 Westbound to Salem St Westbound',
   5: 'From Salem St Westbound to Spring St Southbound',
   6: 'From Spring St Southbound to Riverside Ave Eastbound',
   7: 'From Riverside Ave Eastbound to Locust St Southbound',
   8: 'Locust St Southbound'},
  'route': 'Route 1',
  'x_start': 42.436659000000006,
  'y_start': -71.09007199999999,
  'x_end': 42.41105,
  'y_end': -71.092265,
  'origin': '42.436659000000006, -71.09007199999999',
  'destination': '42.41105, -71.092265',
  'start_zone': 1,
  'end_zone': 2,
  1: ' 7 min',
  2: ' 2.4 miles'},
 {0: 'From 49 Palmer St, Medford, MA 02155, USA to 375 Riverside Ave, Medford, MA 02155, USA',
  

In [50]:
# # if you want use dcitionary instead of a list for keeping the output of the loop so you can make dataframe as below:
# df=pd.DataFrame.from_dict(cengiz,orient='index').T
# df=df.rename(columns ={0:'from', 1: 'time', 2:'distance'}) # change name of the columns
# df

In [51]:
# make a dataframe table by using list which is output of the loop:
df=pd.DataFrame(cengiz)
df=df.rename(columns={0:'from to where',1:'time',2:'distance'}) # change name of the columns
df

,from to where,direction,route,x_start,y_start,x_end,y_end,origin,destination,start_zone,end_zone,time,distance
0,"From 49 Palmer St, Medford, MA 02155, USA to 3...",{0: 'From Highland Ave Eastbound to Fellsway E...,Route 1,42.436659,-71.090072,42.411050,-71.092265,"42.436659000000006, -71.09007199999999","42.41105, -71.092265",1,2,7 min,2.4 miles
1,"From 49 Palmer St, Medford, MA 02155, USA to 3...",{0: 'From Highland Ave Westbound to Elm St Sou...,Route 2,42.436659,-71.090072,42.411050,-71.092265,"42.436659000000006, -71.09007199999999","42.41105, -71.092265",1,2,7 min,3.9 miles
2,"From 12 E Border Cir, Medford, MA 02155, USA t...",{0: 'From E Border Cir Westbound to E Border R...,Route 1,42.439999,-71.088950,42.411045,-71.090880,"42.4399985, -71.0889505","42.41104489999999, -71.0908802",1,2,7 min,3.9 miles
3,"From 12 E Border Cir, Medford, MA 02155, USA t...",{0: 'From E Border Cir Westbound to E Border R...,Route 2,42.439999,-71.088950,42.411045,-71.090880,"42.4399985, -71.0889505","42.41104489999999, -71.0908802",1,2,8 min,2.7 miles
4,"From 2 Nathaniel Rd, Medford, MA 02155, USA to...",{0: 'From Nathaniel Rd Westbound to Cedar Rd N...,Route 1,42.430675,-71.094324,42.406251,-71.094565,"42.4306754, -71.0943239","42.4062507, -71.09456479999999",1,2,6 min,3.6 miles
5,"From 2 Nathaniel Rd, Medford, MA 02155, USA to...",{0: 'From Nathaniel Rd Westbound to Cedar Rd N...,Route 2,42.430675,-71.094324,42.406251,-71.094565,"42.4306754, -71.0943239","42.4062507, -71.09456479999999",1,2,8 min,2.4 miles


In [52]:
# system allows to scrape maximum 149 times for each individual attempt(loop).
# But you can try for new attempts back to back unlimitedly.

df[df['route']=='Route 1'].shape # represent that how many points you could download for this time!

(3, 13)

In [53]:
df.shape

(6, 13)

In [54]:
df.dropna(inplace=True) # null values come from where if there is no option for 'Route 2'
df.shape

(6, 13)

In [20]:
# save the output as csv file
df.to_csv('./Data/csv_data', index=False) 